In [1]:
#import libraries
!pip install scikit-learn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:


# Read the data
df = pd.read_csv("data/Kenya_g4.areaAvgTimeSeries.MYD08_D3_6_1_AOD_550_Dark_Target_Deep_Blue_Combined_Mean.20030101-20221231.180W_90S_180E_90N.csv",
    skiprows=8,
    has_header=True,
)

# Check the first few values in the 'time' column
print("First 10 values in 'time' column:")
print(df.select("time").head(10))

# Step 1: Replace any unwanted characters (like extra spaces or symbols) in the 'time' column
df = df.with_columns(
    pl.col("time").str.replace(r"\s+", "").alias("time_clean")  # Remove all whitespace
)

# Step 2: Manually clean problematic entries like '#NAME?' or empty strings
df = df.with_columns(
    pl.when(pl.col("time_clean").is_in(["#NAME?", ""]))  # Handle invalid entries
    .then(None)
    .otherwise(pl.col("time_clean"))
    .alias("time_clean")
)

# Step 3: Try to parse the cleaned time column (format: mm/dd/yyyy)
df = df.with_columns(
    pl.col("time_clean")
    .str.strptime(pl.Datetime, "%m/%d/%Y")  # Parsing the date in m/d/yyyy format
    .alias("date_parsed")  # Store the parsed date in a new column
)

# Step 4: Display the results to check if the dates are being parsed correctly
print("First 10 rows after parsing 'time' column:")
print(df.select(["time_clean", "date_parsed"]).head(10))

# If parsing is successful, proceed with the aod column
df = df.with_columns(
    pl.when(pl.col("mean_MYD08_D3_6_1_AOD_550_Dark_Target_Deep_Blue_Combined_Mean") == "#NAME?")  # Handle invalid AOD values
    .then(None)
    .otherwise(pl.col("mean_MYD08_D3_6_1_AOD_550_Dark_Target_Deep_Blue_Combined_Mean"))
    .cast(pl.Float64)
    .alias("aod")  # Rename the column to 'aod'.lazy()
)

# Display the final dataframe
print("Final dataframe after processing:")
print(df.head())


TypeError: read_csv() got an unexpected keyword argument 'has_header'